In [7]:
library(rhdf5)
library(Matrix)
library(SingleCellExperiment)

In [16]:
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_cells.h5"

In [17]:
cbind(h5ls(filename)$group,
h5ls(filename)$name)

/,X
/X,data
/X,indices
/X,indptr
/,obs
/obs,__categories
/obs/__categories,batch
/obs/__categories,celltype
/obs/__categories,donor_id
/obs/__categories,pool_id
/obs/__categories,sample_id


In [20]:
ind = h5read(file = filename, "obs")[['_index']] # cell names
var = h5read(file = filename, "var")[['gene_ids-0-0']] # gene names

In [19]:
head(ind)

[1] "AAACCTGAGAGCTATA-1-0-0" "AAACCTGCAATTGCTG-1-0-0" "AAACCTGCAGAGCCAA-1-0-0"
[4] "AAACCTGCATCCCACT-1-0-0" "AAACCTGCATGCCACG-1-0-0" "AAACCTGGTCACCCAG-1-0-0"

In [23]:
head(var)

[1] "ENSG00000243485" "ENSG00000237613" "ENSG00000186092" "ENSG00000238009"
[5] "ENSG00000239945" "ENSG00000237683"

In [24]:
X = h5read(file = filename, "X")
X_data = as.integer(X$data)
X_indptr = as.integer(X$indptr)
X_indices = as.integer(X$indices)
rm(X)

In [27]:
sMat = sparseMatrix(
i = X_indices,
p = X_indptr,
x = X_data,
dims = c(length(var), length(ind)),
index1 = FALSE
)
rownames(sMat) <- as.character(var)
colnames(sMat) <- as.character(ind)

In [29]:
R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_cells.Rds"
saveRDS(sMat, file = R_filename)

In [30]:
obs = h5read(file = filename, "obs")
all_cols = names(obs)
all_cols
categorical_cols = names(h5read(file = filename, "obs/__categories"))
categorical_cols

[1] "__categories"                "_index"                     
 [3] "batch"                       "celltype"                   
 [5] "cluster_id"                  "donor_id"                   
 [7] "log1p_n_genes_by_counts"     "log1p_total_counts"         
 [9] "log1p_total_counts_mito"     "n_counts_all"               
[11] "n_genes_by_counts"           "pct_counts_in_top_100_genes"
[13] "pct_counts_in_top_200_genes" "pct_counts_in_top_500_genes"
[15] "pct_counts_in_top_50_genes"  "pct_counts_mito"            
[17] "pool_id"                     "sample_id"                  
[19] "time_point"                  "total_counts"               
[21] "total_counts_mito"           "treatment"

[1] "batch"      "celltype"   "donor_id"   "pool_id"    "sample_id" 
[6] "time_point" "treatment"

In [32]:
df = data.frame(row.names = obs[["_index"]])
head(df)

AAACCTGAGAGCTATA-1-0-0
AAACCTGCAATTGCTG-1-0-0
AAACCTGCAGAGCCAA-1-0-0
AAACCTGCATCCCACT-1-0-0
AAACCTGCATGCCACG-1-0-0
AAACCTGGTCACCCAG-1-0-0


In [33]:
for (col in all_cols){
#     print(col)
    if (!(col %in% c(categorical_cols, "__categories","index"))){
#         print(col)
#         print(obs[[col]])
        df[[col]] = (as.numeric(unlist(obs[[col]])))   
    }
    if (col %in% c(categorical_cols)){
        colname = paste0("obs/__categories/",col)
        df[[col]] = h5read(file = filename,colname)[((as.numeric(obs[[col]]))+1)]
    }
}

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [34]:
head(df,2)

,_index,batch,celltype,cluster_id,donor_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,...,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,pct_counts_mito,pool_id,sample_id,time_point,total_counts,total_counts_mito,treatment
AAACCTGAGAGCTATA-1-0-0,NA,0,DA,0,HPSI0714i-iudw_1,NaN,NaN,NaN,4370,NaN,...,NaN,NaN,NaN,NaN,pool1,5245STDY7387187,D30,NaN,NaN,NONE
AAACCTGCAATTGCTG-1-0-0,NA,0,DA,0,HPSI0115i-paim_1,NaN,NaN,NaN,4429,NaN,...,NaN,NaN,NaN,NaN,pool1,5245STDY7387187,D30,NaN,NaN,NONE


In [35]:
cData_raw = df[,c("celltype","donor_id","sample_id","pool_id","time_point","treatment")]
cData_raw$index = rownames(cData_raw)
head(cData_raw,2)

,celltype,donor_id,sample_id,pool_id,time_point,treatment,index
AAACCTGAGAGCTATA-1-0-0,DA,HPSI0714i-iudw_1,5245STDY7387187,pool1,D30,NONE,AAACCTGAGAGCTATA-1-0-0
AAACCTGCAATTGCTG-1-0-0,DA,HPSI0115i-paim_1,5245STDY7387187,pool1,D30,NONE,AAACCTGCAATTGCTG-1-0-0


In [36]:
cData <- as.data.frame(cData_raw)
cData_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_cells_cData.Rds"
saveRDS(cData, file = cData_R_filename)

In [ ]:
# build the sce object
sce <- SingleCellExperiment(
assays = list(counts = sMat),
colData = cData)
sce

In [ ]:
sce_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_cells_sce.Rds"
saveRDS(sce, file = sce_R_filename)